In [ ]:
# %help

In [ ]:
# %stop_session

In [ ]:
# %additional_python_modules s3://test-csp-s3-etl-glue-asset/etl_plugin_scripts/leven-1.0.4.tar.gz

# Library declaration

In [ ]:
import os
# from leven import levenshtein  # Levenshtein comparison Cloud
from Levenshtein import distance #  Levenshtein comparison Local
import boto3
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 1000)
pd.reset_option('display.max_rows')


# Function declaration

In [ ]:
def initialize_boto():
    """ Function to initialize boto3 """

    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
    AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN,
    )
    return s3_client


def get_object_from_boto(s3_client, bucket, key):
    """ Function to get the required object from boto given the bucket and the key"""
    
    response = s3_client.get_object(Bucket=bucket, Key=key)
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        # Successful S3 get_object response.
        test_df = pd.read_csv(response.get("Body"), sep='~', dtype=str)
        return test_df
    else: 
        print("Could not get the object response...")
        return None


def create_lev_cols_and_find_the_best_model(test_df_with_nans, sorted_fields_to_check):
    """ Create a column called 'best_model' which tells which model was best per row, based on the lowest score model, (the lowest, the better). """

    columns_to_select = [col for col in sorted_fields_to_check if col in test_df_with_nans.columns]  # Showing only required fields
    test_df_with_nans = test_df_with_nans[columns_to_select]
    print("df with selected columns:")
    display(test_df_with_nans)

    # Cleaning NaN fields in the df
    df = test_df_with_nans.replace(np.nan, '-', regex=True) # As all the CSV was read in string type we don't need the ".astype(str)" at the end of this line
    print("df with nans replaced:")
    display(df)

    # Loop through each pair and calculate distance
    for original_model_col, comparison_model_col, new_col in column_pairs:
        df[new_col] = df.apply(lambda x: distance(x[original_model_col], x[comparison_model_col]), axis=1)  # "levenshtein" for Cloud usage
    
    print("df with levenshtein cols created:")
    display(df)

    cols_kv = [col for col in df.columns if col.startswith('lev_') and col.endswith('_kv')]
    cols_lb = [col for col in df.columns if col.startswith('lev_') and col.endswith('_lb')]

    print(f"cols for kv model: {cols_kv}")
    print(f"cols for kv model: {cols_lb}\n")

    df['sum_kv'] = df[cols_kv].sum(axis=1, numeric_only=True)
    df['sum_lb'] = df[cols_lb].sum(axis=1, numeric_only=True)

    df['best_model'] = np.where(df['sum_kv'] < df['sum_lb'], 'KV model', np.where(df['sum_kv'] > df['sum_lb'], 'LB model', 'TIE'))
    
    print("df with 'best_model' col:")
    display(df)

    print(f"best model summary:")
    model_counts = df['best_model'].value_counts()
    model_counts_df = pd.DataFrame(model_counts)
    model_counts_df.reset_index(inplace=True)
    model_counts_df.columns = ['Model', 'Count']  # Rename columns for clarity

    display(model_counts_df)


    return df


# Pairs of columns to calculate Levenshtein distance between: 
# (Original column, Model to compare, New column name of lev result)
column_pairs = [
    ("Name", "name_kv", "lev_Name_kv"),
    ("Name", "name_lb", "lev_Name_lb"),

    ("Account #", "account #_kv", "lev_Account #_kv"),
    ("Account #", "account #_lb", "lev_Account #_lb"),

    ("FirstName", "firstname_kv", "lev_FirstName_kv"),
    ("FirstName", "firstname_lb", "lev_FirstName_lb"),

    ("LastName", "lastname_kv", "lev_LastName_kv"),
    ("LastName", "lastname_lb", "lev_LastName_lb"),

    ("PrevLastName", "prevlastname_kv", "lev_PrevLastName_kv"),
    ("PrevLastName", "prevlastname_lb", "lev_PrevLastName_lb"),

    ("DOB", "dob_kv", "lev_DOB_kv"),
    ("DOB", "dob_lb", "lev_DOB_lb"),

    ("Phone1", "phone1_kv", "lev_Phone1_kv"),
    ("Phone1", "phone1_lb", "lev_Phone1_lb"),

    ("Customer ID #", "customer id #_kv", "lev_Customer ID #_kv"),
    ("Customer ID #", "customer id #_lb", "lev_Customer ID #_lb")
]

bucket = "test-csp-s3-etl-processing"

unique_fields = list(set(element for tup in column_pairs for element in tup))
sorted_fields_to_check = sorted(unique_fields, key=str.lower)
print(sorted_fields_to_check)

# Levenshtein on one single file example

In [ ]:
key = "ocr/processed/PPH950000/nviPPH950000input20240816124042702276_ocr_kvtesting.csv"
test_csv_path = f"s3://{bucket}/{key}"

In [ ]:
s3_client = initialize_boto()
test_df_with_nans = get_object_from_boto(s3_client, bucket, key)

In [ ]:
csv_cols = set(test_df_with_nans.columns)
possible_cols_to_include = csv_cols - set(sorted_fields_to_check)
possible_cols_to_include

## Levenshtein call

In [ ]:
fields_in_df = sorted_fields_to_check + ['s3_SourcePath', 'RekogResult']
result_df = create_lev_cols_and_find_the_best_model(test_df_with_nans, fields_in_df)

# Levenshtein on multiple files example

In [ ]:
result_csvs = s3_client.list_objects(Bucket=bucket, Prefix="ocr/processed/PPH950000/")

# result_csvs = result_csvs["Contents"][0]["Key"]
file_names = [obj_dict["Key"] for obj_dict in result_csvs["Contents"]]  # Getting the file names in path
print(file_names)
kv_testing_file_names = [file_name for file_name in file_names if "_kvtesting" in file_name]  # Filtering only for testing csvs
print(kv_testing_file_names)

In [ ]:
len(kv_testing_file_names)

In [ ]:
appended_data = pd.DataFrame()

for file_name in kv_testing_file_names:
    test_data = get_object_from_boto(s3_client, bucket, file_name)
    appended_data = pd.concat([appended_data, test_data], axis = 0)

## Levenshtein call

In [ ]:
fields_in_df_multiple = sorted_fields_to_check + ['s3_SourcePath', 'RekogResult']
result_df_multiple_files = create_lev_cols_and_find_the_best_model(appended_data, fields_in_df_multiple)

## Additional analysis

In [ ]:
filtered_df = result_df_multiple_files[result_df_multiple_files['best_model'] == 'KV model']
filtered_df

In [ ]:
result_df_multiple_files.tail(10)